## STAGING

Responsável pelo download, extração e armazenamento dos dados da fonte CEAP.

In [0]:
spark.sql("USE CATALOG mvp_ed_ceap")
spark.sql("USE SCHEMA staging")

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS mvp_ed_ceap.staging.staging_data;

In [0]:
zip_url = "https://www.camara.leg.br/cotas/Ano-2024.csv.zip"
zip_target = "dbfs:/Volumes/mvp_ed_ceap/staging/staging_data/Ano-2024.csv.zip"

dbutils.fs.cp(zip_url, zip_target)

display(dbutils.fs.ls("dbfs:/Volumes/mvp_ed_ceap/staging/staging_data/"))


In [0]:
from pyspark.sql.functions import col

zip_df = (
    spark.read.format("binaryFile")
    .load("dbfs:/Volumes/mvp_ed_ceap/staging/staging_data/Ano-2024.csv.zip")
)

In [0]:
import zipfile
import io

zip_bytes = zip_df.select("content").first()[0]
zip_stream = zipfile.ZipFile(io.BytesIO(zip_bytes))

# descobrir o nome real do CSV dentro do zip
csv_name = [n for n in zip_stream.namelist() if n.lower().endswith(".csv")][0]
csv_bytes = zip_stream.read(csv_name)

csv_target = "dbfs:/Volumes/mvp_ed_ceap/staging/staging_data/Ano-2024.csv"

dbutils.fs.put(csv_target, csv_bytes.decode("utf-8"), overwrite=True)

print("CSV extraído para:", csv_target)


In [0]:
display(dbutils.fs.ls("dbfs:/Volumes/mvp_ed_ceap/staging/staging_data/"))

In [0]:
df = spark.read.option("header", True).csv(
    "/Volumes/mvp_ed_ceap/staging/staging_data/Ano-2024.csv"
)
display(df.limit(5))